<a href="https://colab.research.google.com/github/ghluque/Analisis_BTC/blob/master/copia%20BTC_TX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Primero importamos las librerias que vamos a necesitar 

Requests se utiliza para leer informacion desde una url
json es un tipo de archivo, pandas solo lee json con forma de data frame. 

In [14]:
import pandas as pd
import numpy as np

import requests 
import json 


Nombramos el url de block

In [15]:
block_url="https://blockchain.info/rawblock/0000000000000000000cfe48631b9dc8083201f7bda4deb5bc081e344d1a5913"

block es un diccionario de diccionarios y arrays

In [16]:
block= json.loads(requests.get(block_url).text) 

Por ej: la transaccion 2, que propiedades tiene? 

Lo mas importante de aca es: 
fee: es como una especie de recompensa por transaccion ejecutada que reciben los mineros o encargados de la seguridad del bloque.
hash: Es la "direccion" de la transaccion. 
Inputs: entrada de dinero a "X" billetera.
out=output: salida de dinero a "X" billetera.
prev_ out: salida de dinero de una transaccion previa.
addr: direccion de billetera. 
n: orden de input/output
value: valor en "satoshis" de input/output.


In [4]:
block['tx'][2] 

{'balance': 0,
 'block_height': 619002,
 'block_index': 0,
 'double_spend': False,
 'fee': 100000,
 'hash': '12565431a8f14cc4e65767b76914969474652fa0b69b02e8fcf177f977e9e5d7',
 'inputs': [{'index': 0,
   'prev_out': {'addr': '17A16QmavnUfCW11DAApiJxp7ARnxN5pGX',
    'n': 2,
    'script': '76a91443849383122ebb8a28268a89700c9f723663b5b888ac',
    'spending_outpoints': [{'n': 0, 'tx_index': 0}],
    'spent': True,
    'tx_index': 0,
    'type': 0,
    'value': 253537196},
   'script': '483045022100d981306e2cb0ef59167c408d3dbd81e81af36e9e105ccbf0d110b3d8af39c6f50220246b4e8e7e36be43148778ea6fb8658728535fcdc31503fb821d083f71654d9d0141047146f0e0fcb3139947cf0beb870fe251930ca10d4545793d31033e801b5219abf56c11a3cf3406ca590e4c14b0dab749d20862b3adc4709153c280c2a78be10c',
   'sequence': 4294967295,
   'witness': ''}],
 'lock_time': 0,
 'out': [{'addr': '325kjwqJKbcCnJ2LhjqmmeXaPhYiTZ1EFN',
   'n': 0,
   'script': 'a914044ce4aa9472b406557ae47b52b98d098dc29d7a87',
   'spending_outpoints': [{'n': 1, 't

Maximo de inputs por tx

Maximo de outputs por tx

Agrega 0 a la lista hasta alcanzar el desiredLen

Devuelve una tx de la blockchain como una lista con hash, 5 inputs y 5 outputs.

In [5]:
IN_CNT_MAX=5 
OUT_CNT_MAX=5 
Tx_keys= ['hash','time']
Tx_normalized_cols= Tx_keys + ['i'+str(n)+'v' for n in range(0,IN_CNT_MAX)] + ['o'+str(n)+'v' for n in range(0,OUT_CNT_MAX)]
def pad_list(l,desiredLen):
  
  return l + [0] * (desiredLen-len(l)) if len(l)<desiredLen else l[0:desiredLen]

def tx_in_normalized(tx):
  return [(x['prev_out']['value'] if 'prev_out' in x else 0) for x in tx['inputs']]

def tx_out_normalized(tx):
  return [x['value'] for x in tx['out']]

def tx_normalized(tx):
  
  return [tx[k] for k in Tx_keys] + pad_list(tx_in_normalized(tx),IN_CNT_MAX) + pad_list(tx_out_normalized(tx),OUT_CNT_MAX)

Normalizamos nuestro bloque para que tenga forma de tabla y creamos un dataframe

Convertimos el time del bloque en un date time.

In [6]:
tx= pd.DataFrame( [ tx_normalized(tx) for tx in block['tx'] ], columns= Tx_normalized_cols)

tx.time= pd.to_datetime(tx.time, unit='s') 

Transaccion: x ej si en una anterior compre 3 btc y le pago 1 a Mauricio. Le mando 1 btc a Mauricio y 2 a mi billetera (que seria el vuelto)

In [7]:
print("Direccion anterior {}\nEl vuelto va a {}\n",[ block['tx'][2]['inputs'][0]['prev_out']['addr'], block['tx'][2]['out'][2]['addr'] ])

Direccion anterior {}
El vuelto va a {}
 ['17A16QmavnUfCW11DAApiJxp7ARnxN5pGX', '17A16QmavnUfCW11DAApiJxp7ARnxN5pGX']


Los numeros del tx estan en "satoshis" que son la millonesima parte de 1 btc. Entonces procedemos a dividir por /1000000 para obtener numeros mas legibles

Sumamos para conocer el total en bruto de cada transaccion. Y de paso le agregamos la columna TOTAL al data frame. 

Y por ultimo, mostramos el encabezado.

In [13]:
#imputs
tx['i0v'] /= 1000000
tx['i1v'] /= 1000000
tx['i2v'] /= 1000000
tx['i3v'] /= 1000000
tx['i4v'] /= 1000000


#outputs
tx['o0v'] /= 1000000
tx['o1v'] /= 1000000
tx['o2v'] /= 1000000
tx['o3v'] /= 1000000
tx['o4v'] /= 1000000



tx['TOTAL']=tx.iloc[:,2:11].sum(axis=1)


tx.head()

,hash,time,i0v,i1v,i2v,i3v,i4v,o0v,o1v,o2v,o3v,o4v,TOTAL
0,bfcc1c233827fbf147b9744b58844465abf61232c02fb2...,2020-02-26 00:10:06,0.000000e+00,0.0,0.0,0.0,0.0,1.276409e-27,0.000000e+00,0.000000e+00,0.0,0.0,1.276409e-27
1,8325d3decbb805e09f042fc946bc61076be3578af331b1...,2020-02-26 00:10:06,5.000000e-26,0.0,0.0,0.0,0.0,5.100000e-27,4.489992e-26,0.000000e+00,0.0,0.0,9.999992e-26
2,12565431a8f14cc4e65767b76914969474652fa0b69b02...,2020-02-26 00:10:06,2.535372e-28,0.0,0.0,0.0,0.0,1.180000e-30,7.125764e-30,2.451314e-28,0.0,0.0,5.069744e-28
3,0f24da007c306726713906933e0e171800290cbf5e3047...,2020-02-26 00:10:06,2.451314e-28,0.0,0.0,0.0,0.0,7.875000e-30,9.994323e-30,2.271621e-28,0.0,0.0,4.901629e-28
4,9491757c01ab9c01eb03d6db97e4feba4c4675bf6dd803...,2020-02-26 00:10:06,7.778833e-28,0.0,0.0,0.0,0.0,1.070052e-29,7.671327e-28,0.000000e+00,0.0,0.0,1.555716e-27


He aqui el error. Porque el total no se puede calcular con una simple suma o resta. Sino que hay que identificar los valores de outputs que van a la misma billetera para no operarlos dos veces.

In [ ]:
tx.groupby

KeyError: ignored